<a href="https://colab.research.google.com/github/Songloading/Image-Classfication-Tasks/blob/main/jaxLenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import itertools

import numpy.random as npr

import jax.numpy as np
from jax import jit, grad, random
from jax.experimental import optimizers
from jax.experimental import stax
from jax.experimental.stax import Dense, Relu, LogSoftmax, Conv, MaxPool, Flatten
# from examples import dataset

ImportError: ignored

In [ ]:
# some useful materials:
# http://gcucurull.github.io/deep-learning/2020/04/20/jax-graph-neural-networks/
# https://jax.readthedocs.io/en/latest/_modules/jax/experimental/stax.html
# https://jax.readthedocs.io/en/latest/jax.experimental.stax.html

In [ ]:
def loss(params, batch):
    inputs, targets = batch
    preds = predict(params, inputs)
    return -np.mean(np.sum(preds * targets, axis=1))

In [ ]:
def accuracy(params, batch):
    inputs, targets = batch
    target_class = np.argmax(targets, axis=1)
    predicted_class = np.argmax(predict(params, inputs), axis=1)
    return np.mean(predicted_class == target_class)

In [ ]:
# init_fun: initializes the parameters of the layer.
# apply_fun: forward computation function
init_random_params, predict = stax.serial(
    Conv(out_chan=6, filter_shape=(5, 5), strides=(1, 1), padding="VALID"), Relu,
    MaxPool(window_shape=(2, 2), padding="VALID", spec="NCHW"),
    Conv(out_chan=16, filter_shape=(5, 5), strides=(1, 1), padding="VALID"), Relu,
    MaxPool(window_shape=(2, 2), padding="VALID", spec="NCHW"),
    Flatten,
    Dense(120), Relu,
    Dense(84), Relu,
    Dense(10), LogSoftmax)